# Imports

In [ ]:
import sys

dbutils.widgets.text(name='source_path', defaultValue='.')
source_path = dbutils.widgets.get("source_path")
sys.path.append(source_path)

from modules.utils.spark_session import get_spark
from modules.utils.etl import load_to_silver
from modules.utils.widgets import validate_required_widgets
from scraping_lagallega.transformations.silver import categories, products

# Definitions

In [ ]:
REQUIRED_WIDGETS = validate_required_widgets(
    required_widgets=[
        'target',
        'target_catalog',
        'target_schema',
        'target_tables',
        'source_catalog',
        'source_schema',
        'source_table',
        'partition_by',
        'options'
    ],
    widget_types={
        'target_tables': list,
        'partition_by': list,
        'options': dict
    }
)

In [ ]:
bundle_target = REQUIRED_WIDGETS.get("target", "dev")

target_catalog = REQUIRED_WIDGETS.get("target_catalog", "main")
target_schema = REQUIRED_WIDGETS.get("target_schema", "default")
target_tables = REQUIRED_WIDGETS.get("target_tables", [])
source_catalog = REQUIRED_WIDGETS.get("source_catalog", "main")
source_schema = REQUIRED_WIDGETS.get("source_schema", "default")
source_table = REQUIRED_WIDGETS.get("source_table")
partition_by = REQUIRED_WIDGETS.get("partition_by")
options = REQUIRED_WIDGETS.get("options")

source_catalog = REQUIRED_WIDGETS.get("source_catalog") if bundle_target == 'prod' else f'{source_catalog}_{bundle_target}'
target_catalog = REQUIRED_WIDGETS.get("target_catalog") if bundle_target == 'prod' else f'{target_catalog}_{bundle_target}'

# Load

In [ ]:
for table in target_tables:

    load_to_silver(
        spark=get_spark(),
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=table,
        source_catalog=source_catalog,
        source_schema=source_schema,
        source_table=source_table,
        transform_function=categories.category_transformation if table == 'categories' else products.product_transformation,
        partition_by=partition_by,
        options=options
    )